In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import os
import sys
module_path = os.path.abspath(os.path.join(r'C:\Users\soube\OneDrive\Desktop\Hammudi\Bachelorarbeit\Repository\AP-rent-determination\nn.py'))
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import *


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Selecting the DataSource
dataSource = r"C:\Users\soube\OneDrive\Desktop\Hammudi\Bachelorarbeit\Repository\AP-rent-determination\students_data\cleaned_data_conf_with_IQR_removal.csv"

# Selecting columns to drop out of featureList and creating LabelList
featureDropList = ["_id", "observationDate", "state", "city", "AP_community", "community_id","postcode", "base_rent", "qm2_rent", "DE_qm2_rent"]
LabelList = ["qm2_rent"]

# Create DataFrame from DataSource
try: 
    data = import_data(dataSource)
except:
    data = pd.read_csv(dataSource)
data = data[data["state"] == "Sachsen-Anhalt"]


#data.drop(data.filter(regex = "second"), axis = 1, inplace = True)
#data.drop(data.filter(regex = "third"), axis = 1, inplace = True)

numerical = data.select_dtypes(include=['float16','float32','float64','int8','int16','int32','int64']).columns
#Normalize the data by MinMaxScaling
data.loc[:,numerical] = preprocessing.MinMaxScaler().fit_transform(data.loc[:,numerical])

# Create feature and label lists
y = data[LabelList]
X = data.drop(featureDropList, axis = 1)

y = np.array(y)
X = np.array(X)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD
import keras.backend as K
import seaborn as sns
import keras

Memory usage of dataframe is 79.16 MB
Memory usage after optimization is: 27.23 MB
Decreased by 65.6%


In [12]:
model = Sequential()
model.add(Input(shape = X_train.shape[1]))
model.add(Dense(180, activation = 'relu'))
model.add(Dense(180, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))

loss = 'mse'
metrics = ['mae']
# For regression MSE or MAE are good loss functions
model.compile(optimizer = "adam", loss = loss, metrics = metrics)

In [13]:
from keras.callbacks import Callback
from keras.callbacks import LearningRateScheduler

class LRRangeTuner(Callback):
    def __init__(self, base_lr_range, max_lr_range, step_size):
        self.base_lr_range = base_lr_range
        self.max_lr_range = max_lr_range
        self.step_size = step_size
        self.lr_range_tuner = LearningRateScheduler(self.lr_range_tuner_step)

    def lr_range_tuner_step(self, epoch):
        if epoch % self.step_size == 0:
            lr = K.get_value(self.model.optimizer.lr)
            lr_range = (self.base_lr_range + self.max_lr_range) / 2
            K.set_value(self.model.optimizer.lr, lr_range)
            print('New learning rate:', lr_range)
        return K.get_value(self.model.optimizer.lr)

    def on_epoch_begin(self, epoch, logs=None):
        if epoch % self.step_size == 0:
            self.lr_range_tuner.on_epoch_begin(epoch, logs)

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.step_size == 0:
            self.lr_range_tuner.on_epoch_end(epoch, logs)

# Define the base and maximum learning rate range
base_lr_range = 0.001
max_lr_range = 0.005

# Define the step size for changing the learning rate
step_size = 5

# Instantiate the LRRangeTuner callback
lr_range_tuner = LRRangeTuner(base_lr_range, max_lr_range, step_size)

# Pass the callback to the fit() method when training your model
model.fit(X_train, y_train, callbacks=[lr_range_tuner])


AttributeError: 'NoneType' object has no attribute 'optimizer'